In [1]:
!pip install xgboost==1.6.1
!pip install scikit-optimize


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

choose between xgbr, dtr and ann (1.dtr ,2.xgbr, 3.ann)

In [3]:
#from xgboost import XGBRegressor
#new_model=XGBRegressor()
#new_model.load_model('/content/drive/MyDrive/Colab Notebooks/xgbr_avggc5in.h5')

#pickle for dtr
#import pickle
#with open('/content/drive/MyDrive/Colab Notebooks/rfr_avggc.pkl' , 'rb') as f:
  #  new_model = pickle.load(f)

from keras import models
# Recreate the exact same model, including its weights and the optimizer(r2/3)
new_model = models.load_model('/content/drive/MyDrive/Colab Notebooks/averagegcbayesian4in5out1007r2.h5')
new_model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_35 (Dense)            (None, 120)               600       
                                                                 
 dropout_26 (Dropout)        (None, 120)               0         
                                                                 
 dense_36 (Dense)            (None, 120)               14520     
                                                                 
 dropout_27 (Dropout)        (None, 120)               0         
                                                                 
 dense_37 (Dense)            (None, 90)                10890     
                                                                 
 dropout_28 (Dropout)        (None, 90)                0         
                                                                 
 dense_38 (Dense)            (None, 5)                

In [4]:
import pandas as pd
import numpy as np
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/average GC full dataset new for colab 1007 without powder mass.csv')
Xtest=test.iloc[:,:4].values
Ytest = test.iloc[:,4:].values
print(Xtest[0])


[2.00000000e+00 3.55247982e-01 3.00000000e+00 1.00000000e+03]


In [5]:
#prediction=new_model.predict(Xtest)
#print(prediction)
#print(Ytest)

#import numpy as np

#Defining MAPE function
#def MAPE(Y_actual,Y_Predicted):
 #   mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
  #  return mape

#LR_MAPE= MAPE(Ytest,prediction)
#print("MAPE: ",LR_MAPE)
#print(Ytest[0,2])

In [6]:
#df=pd.DataFrame(prediction,index=None)
#arr=df[2]
#arr2=arr.to_numpy()
#print(arr2)

In [7]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]

def prediction(input):
   
   pred=new_model.predict(input)
   preddf=pd.DataFrame(pred,index=None)
   predvalue=preddf[2]
   predarr=predvalue.to_numpy()


   return predarr


In [11]:
def fitness(input):
    ans=prediction(input)
    return ans
  


In [14]:
target=[(15,10,45)]
weight=[(0.5,0.1,1)]
import random
#generate solutions
input=[]
for s in range(20):
    NPCtype=random.randint(1,2)
    
    #NPCweight=random.randrange(50000,61000,100)/1000
    loading=random.randrange(25,60,1)/100
    KOHconc=random.choice([1,3,0.75])
    #ratio=random.randrange(7,11,1)
    current=random.randrange(100,1101,100)


    input.append([(NPCtype,loading,KOHconc,current)])

input=np.asarray(input)
mating_pool=input.reshape(input.shape[0],input.shape[2])



In [16]:
test=prediction(mating_pool)
test=np.asarray(test)

print(mating_pool)
print(test)

1/1 [==============================] - 0s 16ms/step
[[1.0e+00 3.6e-01 7.5e-01 3.0e+02]
 [2.0e+00 3.5e-01 3.0e+00 2.0e+02]
 [1.0e+00 4.2e-01 7.5e-01 6.0e+02]
 [2.0e+00 2.6e-01 1.0e+00 1.0e+02]
 [1.0e+00 3.5e-01 3.0e+00 2.0e+02]
 [1.0e+00 5.6e-01 1.0e+00 9.0e+02]
 [2.0e+00 5.4e-01 7.5e-01 9.0e+02]
 [2.0e+00 5.5e-01 3.0e+00 5.0e+02]
 [2.0e+00 3.6e-01 7.5e-01 2.0e+02]
 [2.0e+00 3.0e-01 3.0e+00 7.0e+02]
 [1.0e+00 5.7e-01 3.0e+00 4.0e+02]
 [2.0e+00 4.1e-01 7.5e-01 8.0e+02]
 [2.0e+00 5.2e-01 3.0e+00 1.0e+03]
 [1.0e+00 5.6e-01 1.0e+00 5.0e+02]
 [2.0e+00 4.7e-01 1.0e+00 7.0e+02]
 [1.0e+00 3.6e-01 7.5e-01 9.0e+02]
 [1.0e+00 3.9e-01 1.0e+00 8.0e+02]
 [1.0e+00 5.0e-01 1.0e+00 8.0e+02]
 [2.0e+00 2.8e-01 3.0e+00 6.0e+02]
 [2.0e+00 2.7e-01 1.0e+00 9.0e+02]]
[33.460854 20.25098  33.36259  14.357745 23.456    32.521935 32.83834
 31.195654 21.97352  35.3814   32.088646 33.237335 33.269505 34.907303
 34.278534 32.49323  32.655502 32.668865 34.719505 32.863304]


In [17]:
#select first four as parents(best 4)
def parent_selection(mixsort):
    fitness_mating_pool=prediction(mating_pool)
    mix_mating_fitness=np.column_stack((mating_pool,fitness_mating_pool))

    mixsort=mix_mating_fitness[mix_mating_fitness[:, 5].argsort()]
    parents_pool=mixsort[:,0:5]
    parents_results=mixsort[:,5]
    return parents_pool

In [18]:
# crossover(for mating)
def uniform_crossover(A,B,C,D,P):
    for i in range(len(P)):
        if P[i]<0.5:
          temp=A[i]
          A[i]=B[i]
          B[i]=C[i]
          C[i]=D[i]
          D[i]=temp
    return A,B,C,D

def crossover_process(mating_pool):
    A=mating_pool[0]
    B=mating_pool[1]
    C=mating_pool[2]
    D=mating_pool[3]
    P=np.random.rand(4)
    crossover_pool=uniform_crossover(A,B,C,D,P)
    crossover_pool=np.asarray(crossover_pool)
  
    return crossover_pool






In [19]:
#mutation!
def mutation(crossover_pool):
    mutation_rate=random.uniform(0.8,1.2)
    mutation_X=crossover_pool[:,0]
    mutation_pool=crossover_pool[:,1:]*mutation_rate
    final_mutation=np.column_stack((mutation_X,mutation_pool))
    return final_mutation

#mutation_pool=mutation(crossover_pool)
#print(mutation_pool)
#print(mutation_pool)
#mutation_rate=random.uniform(0.98,1.02)
#X=mating_pool[:,0]
#mutation_pool=mating_pool[:,1:]*mutation_rate
#yui=np.column_stack((X,mutation_pool))
#print(mutation_pool,yui)

In [22]:
#survival selection by def
def survivor_selection():
    new_mating_pool=np.insert(mutation(crossover_process(parent_selection(mating_pool))),0,mating_pool,axis=0)
    fitness_new_pool=fitness(new_mating_pool)
    mix_new_fitness=np.column_stack((new_mating_pool,fitness_new_pool))

    mixsortnew=mix_new_fitness[mix_new_fitness[:, 5].argsort()]
    mixsortnew =mixsortnew[::-1]
    mixsortnew=mixsortnew[:]
    new_mating_pool=mixsortnew[0:20,0:5]
    return new_mating_pool


test2=survivor_selection()
print(test2)

1/1 [==============================] - 0s 16ms/step


IndexError: ignored

In [ ]:
def genetic_algorithm():
    parents=parent_selection(mating_pool)
    
    
    cross_p=crossover_process(parents)
   
    mutant_p=mutation(cross_p)
    survived_p=survivor_selection()
    return survived_p
    


print(genetic_algorithm())


In [ ]:
initial_gen=0
max_gen=1000



while initial_gen<max_gen:
   x_new=genetic_algorithm()
   initial_gen+=1 
   mating_pool=x_new
   
   first=x_new[0]
   first=np.asarray(first)
   print([first])
   print("gen",initial_gen,"=",x_new)
   
   test=new_model.predict(x_new)
   c2test=test[:,2]+test[:,4]
   print("best results:",test[0])
   
   print("Fittest C2H4+ethanol:",c2test[0])
   if c2test[0]>=75 : break
   

In [ ]:
print(test[0],mating_pool[0])
print(new_model.predict(mating_pool))
